In [1]:
import argparse
import json
import os   
import shutil
import subprocess
from typing import Any, Dict
import sys
import pandas as pd
from eval_utils import ie_eval
import pathlib
from pathlib import Path
import pandas as pd
import itertools
from eval_utils import relation_matching

root_path = Path("../coviddata")
anno_dir = root_path / "Annotations"
annotator_files = list(anno_dir.glob('*.tsv'))
print(annotator_files)

[PosixPath('../coviddata/Annotations/arezou_annotations.tsv'), PosixPath('../coviddata/Annotations/madeline_annotations.tsv'), PosixPath('../coviddata/Annotations/jeff_annotations.tsv'), PosixPath('../coviddata/Annotations/kristina_annotations.tsv'), PosixPath('../coviddata/Annotations/megan_annotations.tsv')]


In [2]:
golddf = pd.read_csv(annotator_files[1], sep="\t",header=None, names=["id","text","arg0","arg1","rel","y"])
golddf = golddf[golddf["y"]=="accept"]
golddf["rel"][golddf["rel"]=="DO"] ="MECHANISM"
golddf["rel"][golddf["rel"].str.contains("USED")] = "MECHANISM"




In [44]:
annotator_files

[PosixPath('../coviddata/Annotations/arezou_annotations.tsv'),
 PosixPath('../coviddata/Annotations/madeline_annotations.tsv'),
 PosixPath('../coviddata/Annotations/jeff_annotations.tsv'),
 PosixPath('../coviddata/Annotations/kristina_annotations.tsv'),
 PosixPath('../coviddata/Annotations/megan_annotations.tsv')]

In [ ]:
def get_match(pair,labels, text, collapse,jaccard_thresh,match_metric,filter_stop,seen_pred_gold,good_preds):
        if collapse:
            labels = [1,1]
        else:
            labels = [pair[0][2],pair[1][2]]
        m = relation_matching(pair,metric=match_metric, labels = labels,thresh=jaccard_thresh, filter_stop = filter_stop)
        if m and ((i,pair[0][0],pair[0][1],pair[1][0],pair[1][1]) not in seen_pred_gold):
            #good_preds.append([i,pair[0],pair[1]])
            good_preds.append((i,tuple(pair[0]),tuple(pair[1]),text))
            seen_pred_gold[(i,pair[0][0],pair[0][1],pair[1][0],pair[1][1])]=1

In [72]:

jaccard_thresh = 0.3
#iterate over annotator files except gold
for annofile in [a for a in annotator_files if "mad" not in a.name]:
    print(annofile)
    #map USED/DO to MECH
    annodf = pd.read_csv(annofile, sep="\t",header=None, names=["id","text","arg0","arg1","rel","y"])
    annodf = annodf[annodf["y"]=="accept"]
    annodf["rel"][annodf["rel"]=="DO"] ="MECHANISM"
    annodf["rel"][annodf["rel"].str.contains("USED")] = "MECHANISM"
    overlap = set(annodf.id).intersection(set(golddf.id))
    #use only overalpping documents
    annodf = annodf[annodf["id"].isin(overlap)]
    golddf = golddf[golddf["id"].isin(overlap)]

    goldrels = golddf[["id","arg0","arg1","rel"]]#.drop_duplicates()
    goldrels = goldrels.drop_duplicates(subset =["id","arg0","arg1"]).set_index("id")
    if "conf" in annodf.columns:
        predrels = annodf[["id","arg0","arg1","rel","conf"]].drop_duplicates(subset =["id","arg0","arg1"]).set_index("id",inplace=False)
    else:
        predrels = annodf[["id","arg0","arg1","rel"]].drop_duplicates(subset =["id","arg0","arg1"]).set_index("id",inplace=False)
    
    print(len(predrels))
    good_preds1 = []
    seen_pred_gold1 = {}
    good_preds2 = []
    seen_pred_gold2 = {}

    match_count = 0
    for i in predrels.index.unique():
        if i in goldrels.index.unique():
            gold = goldrels.loc[i]
            text = golddf[golddf.id == i]["text"].unique()
            if type(predrels.loc[i]) == pd.core.series.Series:
                preds = [predrels.loc[i].values]
            else:
                preds = predrels.loc[i].values
            c = list(itertools.product(gold.values, preds))
            #for each pair, find gold-annotation matches according to different criteria (e.g., collapse true/false or match_metric jaccard/substring)
            #compute agreement 
            #find the diff between two criteria for each annotator and save to disk
            for pair in c:
                collapse = False
                get_match(pair,labels,text,collapse=collapse,jaccard_thresh=jaccard_thresh,match_metric='substring',filter_stop=True,seen_pred_gold = seen_pred_gold1,\
                good_preds=good_preds1)
                collapse = True
                get_match(pair, labels, text,collapse=collapse,jaccard_thresh= jaccard_thresh,match_metric="substring",filter_stop="True",seen_pred_gold = seen_pred_gold2,\
                good_preds = good_preds2)

    print('collapsed: {0} metric: {1} agree: {2}'.format(True, 'substring',len(seen_pred_gold1)/len(goldrels)))          
    print('collapsed: {0} metric: {1} agree: {2}'.format(False, 'substring',len(seen_pred_gold2)/len(goldrels)))

    diff_csv = []
    for g in good_preds2:
        if tuple(g) not in good_preds1:
            diff_csv.append([g[0],g[3],g[1][0],g[1][1],g[1][2],g[2][0],g[2][1],g[2][2]])
    
    pd.DataFrame(diff_csv,columns = ["abstract_id","text","gold_arg0","gold_arg1","gold_label","anno_arg0","anno_arg1","anno_label"]).to_csv(annofile.name+"_diff_gold.csv",header=True,index=False,sep="\t")

../coviddata/Annotations/arezou_annotations.tsv
68
collapsed: True metric: substring agree: 0.47368421052631576
collapsed: False metric: substring agree: 0.631578947368421
../coviddata/Annotations/jeff_annotations.tsv
77
collapsed: True metric: substring agree: 0.49122807017543857
collapsed: False metric: substring agree: 0.5263157894736842
../coviddata/Annotations/kristina_annotations.tsv
36
collapsed: True metric: substring agree: 0.5087719298245614
collapsed: False metric: substring agree: 0.543859649122807
../coviddata/Annotations/megan_annotations.tsv
66
collapsed: True metric: substring agree: 0.5964912280701754
collapsed: False metric: substring agree: 0.6842105263157895
